In [1]:
import pandas as pd
import requests


def stock_telegraph_cls(symbol: str = "全部") -> pd.DataFrame:
    """
    财联社-电报
    https://www.cls.cn/telegraph
    :param symbol: choice of {"全部", "重点"}
    :type symbol: str
    :return: 财联社-电报
    :rtype: pandas.DataFrame
    """
    session = requests.session()
    url = "https://m.cls.cn/telegraph"
    session.get(url)  # 获取 cookies
    params = {
        "refresh_type": "1",
        "rn": "10",
        "last_time": "",
        "app": "CailianpressWap",
        "sv": "1",
    }
    ts = pd.Timestamp(pd.Timestamp.now())
    current_time = 1388425427 #int(ts.timestamp())
    params.update({"last_time": current_time})
    url = "https://m.cls.cn/nodeapi/telegraphs"
    r = session.get(url, params=params)
    data_json = r.json()
    temp_df = pd.DataFrame(data_json["data"]["roll_data"])
    next_time = temp_df["modified_time"].values[-1]
    n = 1
    big_df = temp_df.copy()
    while n < 20:
        params.update({"last_time": next_time})
        r = session.get(url, params=params)
        data_json = r.json()
        temp_df = pd.DataFrame(data_json["data"]["roll_data"])
        big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)
        next_time = temp_df["modified_time"].values[-1]
        n += 1
    big_df = big_df[["title", "content", "ctime", "level",'subjects']]
    big_df["ctime"] = pd.to_datetime(
        big_df["ctime"], unit="s", utc=True
    ).dt.tz_convert("Asia/Shanghai")
    big_df.columns = ["标题", "内容", "发布时间", "等级",'主题']
    big_df.sort_values(["发布时间"],ascending=False, inplace=True)
    big_df.reset_index(inplace=True, drop=True)
    big_df["发布日期"] = big_df["发布时间"].dt.date
    big_df["发布时间"] = big_df["发布时间"].dt.time

    
    
    def extract_subject(subjects):
        l = []
        if subjects != None:
            for s in subjects:
                subject_name = s['subject_name']
                l.append(subject_name)
        return l


    big_df["主题"] = big_df['主题'].apply(extract_subject)   
    
    big_df = big_df.drop_duplicates(subset=["标题", "发布日期", "发布时间"])
    
    
    if symbol == "重点":
        big_df = big_df[(big_df['等级'] == 'B') | (big_df['等级'] == 'A')]
        big_df.reset_index(inplace=True, drop=True)
        big_df = big_df[["主题","标题", "内容", "发布日期", "发布时间"]]
        return big_df
    else:
        big_df = big_df[['等级',"主题","标题", "内容", "发布日期", "发布时间"]]
        return big_df


stock_telegraph_cls_df = stock_telegraph_cls(symbol="全部")
stock_telegraph_cls_df[(stock_telegraph_cls_df['等级'] == 'B') | (stock_telegraph_cls_df['等级'] == 'A')]

KeyError: 'modified_time'

In [70]:
stock_telegraph_cls_df[(stock_telegraph_cls_df['等级'] == 'B') | (stock_telegraph_cls_df['等级'] == 'A')]

,等级,主题,标题,内容,发布日期,发布时间
12,B,[],吴晓求称资本市场不能仅以融资为目标,【吴晓求称资本市场不能仅以融资为目标】财联社12月23日电，在“粤港澳大湾区金融之夜”活动现场，中国人民大学国家金融研究院院长吴晓求做了题为《金融强国的实现路径》的主旨演讲。他提出，建设金融强国，意味着中国资本市场要逐渐成为全球重要的财富管理中心，当前距离这个目标还有较大的差距。要实现金融强国的目标，首先要改变资本市场就是融资市场的理念，如果把资本市场简单定义为融资市场，那么这个市场很难成长。资本市场应该是一个财富管理的市场，必须对现行的更有利于融资的制度、政策、规则进行改革，完善相关制度，使其从融资市场转型为投资市场。 (央视财经),2023-12-23,22:27:17
26,B,"[期货市场情报, 航运]",85%的集装箱运输公司暂停红海航线 行业再现运价跳涨、一舱难求,【85%的集装箱运输公司暂停红海航线 行业再现运价跳涨、一舱难求】财联社12月23日电，上海航运交易所发布了最新一周的中国出口集装箱运输市场的周度报告显示，受红海地区受到武装袭击的威胁，亚欧等航线运输市场运价大幅上涨，带动综合指数上涨14.8%至1254.99点。该报告的数据显示，上海港出口至欧洲、地中海基本港市场运价已经达到1497美元/TEU（20英尺集装箱）和2054美元/TEU（40英尺集装箱），波斯湾航线的运价也达到了1477美元/TEU。据宁波航运交易所12月22日发布的报告显示，目前85%的集装箱船班轮公司均已通知暂停红海航线货物的接载。受此影响，少数开航班次一舱难求，市场订舱运价大幅跳涨，宁波航运交易所发布的红海航线指数也较上周上涨161.9%至3649.5点。,2023-12-23,20:14:28
27,B,"[汽车大新闻, 半导体芯片, 智能驾驶]",蔚来发布首颗自研智能驾驶芯片神玑NX9031,【蔚来发布首颗自研智能驾驶芯片神玑NX9031】财联社12月23日电，在今日举办的2023NIO DAY上，蔚来汽车发布首颗自研智能驾驶芯片——神玑NX9031。神玑NX9031采用5nm车规工艺制程，有超过500亿颗晶体管。蔚来期望用一颗自研芯片实现目前业界四颗旗舰智能驾驶芯片的性能，使得效率和成本更优。（蓝鲸记者 徐昊）,2023-12-23,20:06:24
28,B,[],12月23日周六《新闻联播》要闻21条,【12月23日周六《新闻联播》要闻21条】财联社12月23日电，今天《新闻联播》主要内容有：\n1、【新思想引领新征程】厚植高质量发展绿色底色 生态文明建设迈上新台阶；\n2、王沪宁会见中国基督教第十一次代表会议代表；\n3、秋粮收购超一亿吨 收购进度过半；\n4、生产供给稳步回升 装备制造业加速向好；\n5、【新时代新征程新伟业】冰雪经济赋能新疆乡村振兴；\n6、抗震救灾各项工作紧张有序进行；\n7、【党旗在基层一线高高飘扬】争分夺秒搜救 为了生命坚守；\n8、今年我国规范命名15.6万个乡村地名；\n9、增发国债第二批项目清单下达；\n10、160个国家乡村振兴重点帮扶县贷款发放年均增长14.5%；\n11、2024年全国硕士研究生招生考试今日开考；\n12、2023年西藏民生投入132.8亿元；\n13、中央广播电视总台超高清示范园“央视界”建设启动；\n14、内蒙古引绰济辽输水工程隧洞段全线贯通；\n15、全国新发职业病确诊病例呈下降趋势；\n16、联合国通过决议将春节作为联合国假日；\n17、以军向加沙地带中部发起新攻势 哈马斯武装派别袭击以军运兵车 联合国安理会通过决议要求加大对加沙地带人道援助力度；\n18、俄称与乌军在第聂伯河左岸交战 乌称击落俄军战机 俄副外长称俄美关系已陷入“昏迷”状态；\n19、美国2024财年军费预算再达历史新高；\n20、日修改武器出口原则 出口致命成品武器；\n21、欧洲多国遭风暴袭击 2死多伤。,2023-12-23,19:58:01
56,B,"[游戏行业头条, 游戏]",国家新闻出版署：《网络游戏管理办法》（征求意见稿）旨在促进行业繁荣健康发展 将听取各方意见进一步修改完善,【国家新闻出版署：《网络游戏管理办法》（征求意见稿）旨在促进行业繁荣健康发展 将听取各方意见进一步修改完善】财联社12月23日电，据中国新闻出版广电报，12月22日，国家新闻出版署起草的《网络游戏管理办法》（征求意见稿）向社会公开征求意见。国家新闻出版署有关负责人表示，征求意见稿立足于保障和促进网络游戏行业繁荣健康发展，对解决网络游戏经营单位准入等问题作了明确，设立了“保障与奖励”专章，提出一系列鼓励措施。同时，对保护未成年人和消费者权益作了规定。征求意见稿起草的过程中，通过多种方式广泛听取了相关部门、行业协会、企业等各方意见。该负责人表示，部门规章向社会公开征求意见是更广泛听取意见、完善规章条款的过程。对于各方就征求意见稿第十七条、第十八条及其他一些内容提出的关切和意见，国家新闻出版署将认真研究，并将在继续听取相关部门、企业、用户等各方意见的基础上进一步修改完善。,2023-12-23,17:28:32
65,B,[],《科创板日报》评选2023年十大科技热词,【《科创板日报》评选2023年十大科技热词】过去的一年中，国际形势波谲云诡，海外无风险收益率持续上行压制医药、科技公司估值。但与此同时，“下一个十年”式的宏大叙事层出不穷，一次次撩拨市场投资者的心弦。回首这一年，人工智能浪潮席卷全球，AI大模型遍地开花，各行各业都迎来深刻变革。这场由ChatGPT掀起的浪潮，也在A股激起阵阵涟漪——光模块、AI应用、算力芯片、HBM无不与之密切相关。人形机器人则作为AI的具象载体被寄托了无限的遐想，由此衍生出减速器、传感器、peek材料等一系列细分概念。\n\n但，市场的热情终究会冷却。潮水退去，面对远处未知的风浪，最牢固的船才能航行得最远。总结过去，是为了更好展望未来。当我们在回溯每一个2023年科技热词的时候，也是在重新审视当时的市场审美和产业逻辑，以求得到一丝启发。为此，《科创板日报》评选出2023年十大科技热词。你看好哪些概念穿越时间继续远航？又或者是，明年会有哪些后浪拍死前浪？欢迎留下您的评论。,2023-12-23,16:53:35
67,B,[眼科医疗],贵港市纪委监委：网传爱尔眼科医院“中秋外联明细表”图片引关注 已成立调查组,【贵港市纪委监委：网传爱尔眼科医院“中秋外联明细表”图片引关注 已成立调查组】财联社12月23日电，贵港市纪委监委发布情况通报：近日，网传贵港爱尔眼科医院“中秋外联明细表”图片的消息引发网民关注，我委对此高度重视，已成立调查组，依规依纪依法对相关情况开展核查处理。近日，网友急诊向日葵艾芬发帖称，贵港爱尔眼科医院医生在手术台上用拳头猛击患者头部，该事件引发网友关注。12月21日，该博主再次爆料，发布了两张标为“中秋外联明细表”的图片，其中涉及42名公职人员以及59000元额度的购物卡，并在评论中透露上述图片的涉事单位为贵港爱尔眼科医院。,2023-12-23,16:32:35
112,B,[环球市场情报],联合国通过决议将春节作为联合国假日,【联合国通过决议将春节作为联合国假日 】财联社12月23日电，第78届联合国大会22日协商一致通过决议，将春节（农历新年）确定为联合国假日。\n\n中国常驻联合国代表团临时代办戴兵大使指出，中国推动春节成为联合国假日，是践行全球文明倡议、倡导尊重世界文明多样性的务实行动。春节正式成为联合国假日，充分展现了中华文明的传播力影响力，将有力促进世界不同文明的交流互鉴，积极体现联合国倡导的多元、包容文化价值理念。联大决议将春节确定为联合国假日，得到了联合国会员国和联合国秘书处工作人员的热烈欢迎和踊跃支持。,2023-12-23,09:41:56
116,B,[],国家发展改革委：增发国债第二批项目清单下达 累计安排金额超8000亿元,【国家发展改革委：增发国债第二批项目清单下达 累计安排金额超8000亿元】财联社12月23日电，据国家发展改革委官微消息，近日，国家发展改革委商有关部门下达2023年增发国债第二批项目清单，共涉及增发国债项目9600多个，拟安排国债资金超5600亿元。截至目前，前两批项目涉及安排增发国债金额超8000亿元，1万亿元增发国债已大部分落实到具体项目。下一步，国家发展改革委将按照党中央、国务院决策部署，会同有关部门督促各地高质量推进项目建设，加快资金拨付使用，尽快形成实物工作量。同时，加强项目监管，着力把项目建设成为民心工程、优质工程、廉洁工程。,2023-12-23,09:11:26
134,B,"[环球市场情报, 港股动态]",周六你需要知道的隔夜全球要闻,【周六你需要知道的隔夜全球要闻】\n1、美股三大指数收盘涨跌不一，均录得周线八连涨。纳指涨0.19%，本周累涨1.21%；标普500指数涨0.17%，本周累涨0.75%；道指跌0.05%，本周累涨0.22%。欧洲主要股指收盘涨跌不一，德国DAX30指数涨0.14%。\n2、以色列国防军称几乎完全控制加沙地带北部。\n3、亚特兰大联储GDPNow模型预计美国第四季度GDP增速为2.3%，此前预计为2.7%。\n4、美国消费者对未来一年通胀预期降至2021年来最低水平。\n5、美国白宫经济顾问布雷纳德表示，美国通胀率正在持续下降。\n6、普京将参加欧亚经济委员会最高理事会会议及独联体国家领导人非正式会议。\n7、俄罗斯外长与伊朗外长通电话，双方讨论了有关南高加索地区的最新事态发展，并对加沙局势等

In [8]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
session = requests.session()
url = "https://m.cls.cn/telegraph"
session.get(url)  # 获取 cookies
params = {
    "refresh_type": "1",
    "rn": "10",
    "last_time": "",
    "app": "CailianpressWap",
    "sv": "1",
}
ts = pd.Timestamp(pd.Timestamp.now())
current_time = 1609350227 #int(ts.timestamp())
params.update({"last_time": current_time})
url = "https://m.cls.cn/nodeapi/telegraphs"
r = session.get(url, params=params)
data_json = r.json()

In [11]:
data_json

{'error': 0, 'data': {'roll_data': []}}

In [61]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def extract_subject(subjects):
    l = []
    if subjects != None:
        for s in subjects:
            subject_name = s['subject_name']
            l.append(subject_name)
    return l


pd.DataFrame(pd.DataFrame(data_json['data']['roll_data'])['subjects'].apply(extract_subject))

,subjects
0,"[期货市场情报, 天气变化, 厄尔尼诺现象]"
1,"[期货市场情报, 地震快讯]"
2,[期货市场情报]
3,[环球市场情报]
4,"[期货市场情报, 地震快讯]"
5,"[环球市场情报, 原油市场动态, 期货市场情报]"
6,[数字经济]
7,[]
8,[]
9,[区块链]


In [29]:
d = {'is_reporter_subject': False, 'article_id': 1552556, 'plate_id': 88, 'attention_num': 0, 'is_attention': False, 'subject_description': '', 'subject_id': 1501, 'subject_img': '', 'category_id': 0, 'subject_name': '期货市场情报'}

In [48]:
pd.DataFrame.from_dict(d, orient='index').T['subject_name']

0    期货市场情报
Name: subject_name, dtype: object

In [45]:
type(json.loads(json.dumps(d)))

dict

In [53]:
d['subject_name']

'期货市场情报'